## Final project

Find the solution of 

$$
-\Delta \mathbf u + \mathbf u = \mathbf f 
$$

on a cube $[0,1]^3$. For the sake of simplicity you can consider just **one finite element** and homogeneous Neumann boundary conditions. 

Consider Lagrangian finite elements, with Chebyshev nodes, of high order (up to 30--40).

The above problem can be rewritten in matrix form as

$$
(A+M)\mathbf u = \mathbf f
$$

where $A$ is the stiffness matrix, and $M$ is the mass matrix.



1 - Assemble the matrices and solve the system with a **direct solver**. Plot (loglog) the $L_2$ norm of the error with respect to a known exact solution as function of the degree of the finite element. 

2 - Use an iterative solver with a chosen preconditioner. Compare and comment the computational burden between the direct and iterative case.

3 - **OPTIONAL** Using the iterative solver, implement a matrix-free approach where the computation of the matrices is substituted with a  function which returns the *matrix-vector* product. In doing so, you must take care of rewriting the assemble of the matrix as a sum of many *matrix-matrix* products exploiting all the possible tensor products. 

Compare and comment the compuational cost of 100 matrix-vector product when the matrix is assembled and when it is computed on-the-fly as a function of the degree of the finite element.